# 코스미 데이터 분석

In [1]:
import os
import pandas as pd
import ast # to convert a string to dict
from collections import Counter

C:\Users\dongm\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
mydir = 'C:/Users/dongm/COSMOCHAIN/Data/COSME/'
filelist =os.listdir(mydir)
display(filelist)

['comments.csv',
 'comments_update.csv',
 'downvote.csv',
 'posts.csv',
 'posts_tags.csv',
 'recomments.csv',
 'recomments_update.csv',
 'sort.csv',
 'user.csv',
 'vote.csv']

# csv 파일에서 dataframe으로 전환!

In [3]:
# status 공란 비우기
def getCleanDF(mydir,inp):
    try:
        data = pd.read_csv(mydir+inp, encoding='UTF-8')
    except:
        data = pd.read_csv(mydir+inp, encoding='cp949')

    data = data.dropna()
    data=data[data.iloc[:,0]!='""']
    try:
        data=data[data.iloc[:,1]!='""']
    except:
        pass
    
    try:
        data=data[data.iloc[:,2]!='""'] # ?
    except:
        pass
    return data


In [4]:
dic_data={}
for i in range(len(filelist)):
    inputfilename= filelist[i]
    df=getCleanDF(mydir,inputfilename)
    dic_data[inputfilename]=df

C:\Users\dongm\Anaconda3\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


# tag + activities

In [5]:
activities = ['recomments.csv', 'comments.csv', 'downvote.csv', 'vote.csv', 'posts.csv']
def tags_activity(activity):
    caller = dic_data[activity]
    other  = dic_data['posts_tags.csv']
    try:
        tags_activity = caller.set_index('postid').join(other.set_index('postid'))
    except:
        tags_activity = caller.set_index('targetid').join(other.set_index('postid'))
        tags_activity.index.name = 'postid'
        
    return tags_activity

In [6]:
tags_recomments = tags_activity('recomments.csv')
tags_comments = tags_activity('comments.csv')
tags_downvote = tags_activity('downvote.csv')
tags_vote = tags_activity('vote.csv')
tags_posts = tags_activity('posts.csv')

# what is category of this tag?

In [7]:
sortoftag = dic_data['sort.csv']
sortoftag.head()

,태그,횟수,분류
0,코스미,7874,기타
1,보습,3453,기능
2,향수,2803,카테고리별
3,메이크업,2241,메이크업
4,립스틱,2104,카테고리별


# user

In [8]:
user = dic_data['user.csv']
user = user.reset_index()
user = user.drop(columns=['index'])

skinConcerns = []
skinTone = []
skinType = []
# get Age of user (2019년 기준)
age = []
gender   = []
for i in range(len(user)):
    skinConcerns.append(ast.literal_eval(user['basicskininfo'][i])['skinConcerns'])
    skinTone.append(ast.literal_eval(user['basicskininfo'][i])['skinTone'])
    skinType.append(ast.literal_eval(user['basicskininfo'][i])['skinType'])
    thisage = 2020 - int(ast.literal_eval(user['profileproperties'][i])['birthday'][:4])
    age.append(thisage)
    gender.append(ast.literal_eval(user['profileproperties'][i])['gender'])
    

user['skinConcerns']= skinConcerns
user['skinTone']    = skinTone
user['skinType']    = skinType
user['age']         = age
user['gender']      = gender

user = user.drop(columns=['basicskininfo','profileproperties'])
user.head()

,userid,skinConcerns,skinTone,skinType,age,gender
0,851e3305-731c-47fa-9490-9f5ad27bb5e0,[DRY],BRIGHT,OILY,43,MALE
1,57e1cd81-1e3a-43a4-a29b-87afdec001b2,"[WHITENING, SENSITIVE]",BRIGHT,DRY,39,MALE
2,8f589409-f23a-49ef-bd24-d5883793d8e2,"[WRINKLE, BLACKHEAD, DRY]",DARK,OILY,35,MALE
3,5abdfe80-2caa-4981-b117-5500a085f9d5,"[WRINKLE, SENSITIVE, DRY]",DARK,COMBINATION,37,MALE
4,ee9d7f9a-2dc9-40f1-99eb-57e611ad7cc3,"[WHITENING, BLACKHEAD, ACNE]",MIDDLE,OILY,24,FEMALE


# Now we have

In [9]:
user
tags_recomments
tags_comments
tags_downvote
tags_vote
tags_posts
sortoftag
print('Tables are ready!')

Tables are ready!


# get result1

In [10]:
def user_activity(caller,other):
    try:
        user_act = caller.set_index('userid').join(other.set_index('authorid'))
    except:
        user_act = caller.set_index('userid').join(other.set_index('userid'))

    user_act = user_act.dropna()
    user_act.index.name = 'userid'
    return user_act

In [11]:
caller = user
activities = [tags_recomments, tags_comments, tags_downvote, tags_vote, tags_posts]
other  = tags_posts

user_posts = user_activity(caller,other)

print('skinType:',set(user_posts['skinType']))
print('skinTone:',set(user_posts['skinTone']))

skinType: {'OILY', 'DRY', 'NORMAL', 'COMBINATION'}
skinTone: {'MIDDLE', 'BRIGHT', 'DARK'}


In [12]:
def seeTopN(df, colname, col):
    user = df[df[colname]==col]
    counter = Counter(user['name'])
    '''
    ignore = ['코스미','보습','향수','메이크업','핸드크림','이니스프리','코스미콘테스트','코스미화이팅']
    for word in list(counter):
        if word in ignore:
            del counter[word]
    '''
    
    return counter.most_common(30)


In [13]:
om = seeTopN(user_posts, 'skinTone', 'MIDDLE')
ob = seeTopN(user_posts, 'skinTone', 'BRIGHT')
od = seeTopN(user_posts, 'skinTone', 'DARK')

yd = seeTopN(user_posts, 'skinType', 'DRY')
yc = seeTopN(user_posts, 'skinType', 'COMBINATION')
yn = seeTopN(user_posts, 'skinType', 'NORMAL')
yo = seeTopN(user_posts, 'skinType', 'OILY')



yd = dict(yd)
yc = dict(yc)
yn = dict(yn)
yo = dict(yo)

c1=Counter(list(yd.keys()))
c2=Counter(list(yc.keys()))
c3=Counter(list(yn.keys()))
c4=Counter(list(yo.keys()))
ccc=c1+c2+c3+c4
keylist=[]
for i,key in enumerate(ccc):
    if ccc[key]>3:
        keylist.append(key)
        
print('톤별:', keylist)        
for kl in keylist:
    del yd[kl]
    del yc[kl]
    del yn[kl]
    del yo[kl]
    

om = dict(om)
od = dict(od)
ob = dict(ob)

c1=Counter(list(om.keys()))
c2=Counter(list(ob.keys()))
c3=Counter(list(od.keys()))
ccc=c1+c2+c3
keylist=[]
for i,key in enumerate(ccc):
    if ccc[key]==3:
        keylist.append(key)
        
print('톤별:', keylist)        
for kl in keylist:
    del om[kl]
    del ob[kl]
    del od[kl]

톤별: ['코스미', '보습', '메이크업', '향수', '립스틱', '수분', '핸드크림', '틴트', '크림', '마스크팩', '수분크림', '립밤', '코스미콘테스트', '로션', '스킨', '에센스', '샴푸', '이니스프리', '선크림', '바디로션', '향수추천', '코스미화이팅']
톤별: ['코스미', '보습', '향수', '메이크업', '핸드크림', '립스틱', '수분', '틴트', '마스크팩', '이니스프리', '수분크림', '립밤', '크림', '로션', '샴푸', '에센스', '코스미콘테스트', '선크림', '바디로션', '코스미화이팅', '스킨', '미스트', '향수추천']


In [14]:
display('DRY:',yd)
print('-------------------------------------')
display('COMBINATION:',yc)
print('-------------------------------------')
display('NORMAL:',yn)
print('-------------------------------------')
display('OILY:',yo)
print('-------------------------------------')
print('-------------------------------------')

display('BRIGHT:',ob)
print('-------------------------------------')
display('MIDDLE:',om)
print('-------------------------------------')
display('DARK  :',od)

'DRY:'

{'립스틱추천': 212,
 '바디워시': 199,
 '틴트추천': 187,
 '파운데이션': 177,
 '썬크림': 156,
 '블러셔': 154,
 '미백': 151,
 '토너': 150}

-------------------------------------


'COMBINATION:'

{'섀도우': 354,
 '틴트추천': 302,
 '립스틱추천': 293,
 '블러셔': 289,
 '올리브영': 254,
 '파운데이션': 249,
 '미스트': 234,
 '토너': 226}

...

'MIDDLE:'

{'바디워시': 443,
 '섀도우': 434,
 '미백': 380,
 '틴트추천': 358,
 '블러셔': 358,
 '립스틱추천': 350,
 '파운데이션': 328}

-------------------------------------


'DARK  :'

{'바디워시': 163,
 '미백': 125,
 '토너': 116,
 '썬크림': 114,
 '클렌징': 113,
 '남자향수': 108,
 '여드름': 94}

In [15]:
def groupdf(df, col):
    gb_df = df.groupby([col])
    gb_df=[gb_df.get_group(x) for x in gb_df.groups]
    return gb_df